# Prompt Templates

Can find more prompt templates here: https://smith.langchain.com/hub/hwchase17/react

# LangGraph

A lot of the agentic frameworks from research papers could be represented as DAGs with:
- nodes: agents or functions
- edges: connect nodes
- conditional edges: decisions

Data/State is important in LangGraph
- it is accessible to all parts of the graph
- local to the graph
- can be stored in a persistence layer

# Agent LangGraph Implementation

## Setup

In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
# used to graph
from langgraph.graph import StateGraph, END
# used to construct agent state
from typing import TypedDict, Annotated 
import operator
# message types to represent human and ai messages
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
# standard interface in LangChain - can swap language models easily
from langchain_openai import ChatOpenAI
# tavily search engine for tool
from langchain_community.tools.tavily_search import TavilySearchResults

## Tools

In [3]:
tool = TavilySearchResults(max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


## Agent State

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

## Agent

In [5]:
class Agent:
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END} # Routing logic
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        # Turn the graph into a lang chain runnable
        self.graph = graph.compile()
        self.tools = { t.name: t for t in tools }
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        # because we annotated the type the following does not override "messages"
        # but adds to it
        return { "messages": [message] }
    
    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result)))
        print("Back to the model")
        return { "messages": results }

## Usage

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt)

### Graph Visualization

In [8]:
from IPython.display import Image

# Image(abot.graph.get_graph().draw_png())

### Questions

In [9]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_ufoLDwMhNA4PDhyP9vLp3iBT'}
Back to the model


In [10]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ufoLDwMhNA4PDhyP9vLp3iBT', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 153, 'total_tokens': 175, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-01c8d7d2-b989-4d34-b020-b4ee75b63bd9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_ufoLDwMhNA4PDhyP9vLp3iBT'}]),
  ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'na

In [11]:
result['messages'][-1].content

'The current weather in San Francisco is partly cloudy with a temperature of 55.9°F (13.3°C). The wind speed is 5.4 mph (8.6 kph) coming from the west-southwest direction. The humidity is at 90%, and the visibility is 9.0 miles.'

In [12]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_ZzWaCtrtNXZS4ecGn5Oahz6j'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_qall2KAo3h4eascx6hVY1eu3'}
Back to the model


In [14]:
print(result['messages'][-1].content)

The current weather in San Francisco is partly cloudy with a temperature of 55.9°F (13.3°C). The wind speed is 5.4 mph and the humidity is at 90%.

In Los Angeles, the current weather is also partly cloudy with a temperature of 57.0°F (13.9°C). The wind speed is 4.3 mph and the humidity is at 77%.


In [15]:
query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': 'call_rnqf4gaGDfEDlSnkEKaq7MC9'}
Back to the model
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Kansas City Chiefs headquarters location'}, 'id': 'call_2pr8R39EWcJKtKzp1dse3LDY'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Missouri GDP 2024'}, 'id': 'call_VAgGjjutJRUchGjrcgt30bRr'}
Back to the model


In [16]:
print(result['messages'][-1].content)

1. The Kansas City Chiefs won the Super Bowl in 2024.

2. The headquarters of the Kansas City Chiefs is located in Kansas City, Missouri.

3. As of 2023, the GDP of Missouri was $348.5 billion.
